In [1]:
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from statsmodels.multivariate.manova import MANOVA
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns

# (1) Chuẩn bị và tiền xử lý dữ liệu

In [ ]:
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  
data = pd.read_csv(file_path)

Saving df_selected_output.csv to df_selected_output.csv


In [ ]:
columns_needed = [
    'Customer Id',         # Mã khách hàng
    'Order Id',             # Mã đơn hàng
    'order date (DateOrders)',           # Ngày đặt hàng
    'Order Item Total',     # Tổng giá trị đơn hàng
    'Days for shipping (real)',    # Số ngày vận chuyển thực tế
    'Customer Segment',     # Loại khách hàng (Consumer, Corporate, Home Office)
    'Category Name',        # Tên danh mục sản phẩm
    'Shipping Mode'         # Phương thức vận chuyển
]

df_selected = data[columns_needed]
print("✅ Dữ liệu sau khi chọn cột:")
print(df_selected.head())
df_selected.info()

✅ Dữ liệu sau khi chọn cột:
   Customer Id  Order Id order date (DateOrders)  Order Item Total  \
0        20755     77202           1/1/2015 0:00        314.640015   
1        19492     75939           1/1/2015 0:21        311.359985   
2        19491     75938           1/1/2015 0:21        309.720001   
3        19490     75937           1/1/2015 0:21        304.809998   
4        19489     75936           1/1/2015 1:03        298.250000   

   Days for shipping (real) Customer Segment   Category Name   Shipping Mode  
0                         3         Consumer  Sporting Goods  Standard Class  
1                         5         Consumer  Sporting Goods  Standard Class  
2                         4         Consumer  Sporting Goods  Standard Class  
3                         3      Home Office  Sporting Goods  Standard Class  
4                         2        Corporate  Sporting Goods  Standard Class  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518

In [ ]:
print("🔍 Checking missing values:")
print(df_selected.isnull().sum())

df_selected = df_selected.drop_duplicates()

🔍 Checking missing values:
Customer Id                 0
Order Id                    0
order date (DateOrders)     0
Order Item Total            0
Days for shipping (real)    0
Customer Segment            0
Category Name               0
Shipping Mode               0
dtype: int64


# (2) Xây dựng mô hình LRFM

In [6]:
print(df_selected.columns.tolist())

['Customer Id', 'Order Id', 'order date (DateOrders)', 'Order Item Total', 'Days for shipping (real)', 'Customer Segment', 'Category Name', 'Shipping Mode']


In [7]:
df_selected = df_selected.rename(columns={
    'Customer Id': 'CustomerID',
    'Order Id': 'OrderID',
    'order date (DateOrders)': 'OrderDate',
    'Order Item Total': 'OrderTotal',
    'Days for shipping (real)': 'DaysShipping',
    'Customer Segment': 'Segment',
    'Category Name': 'Category',
    'Shipping Mode': 'ShipMode'
})

print(df_selected.info())

<class 'pandas.core.frame.DataFrame'>
Index: 180518 entries, 0 to 180518
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CustomerID    180518 non-null  int64  
 1   OrderID       180518 non-null  int64  
 2   OrderDate     180518 non-null  object 
 3   OrderTotal    180518 non-null  float64
 4   DaysShipping  180518 non-null  int64  
 5   Segment       180518 non-null  object 
 6   Category      180518 non-null  object 
 7   ShipMode      180518 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 12.4+ MB
None


In [8]:
df_selected['OrderDate'] = pd.to_datetime(df_selected['OrderDate'], errors='coerce')

In [ ]:
df_selected['OrderDate'] = pd.to_datetime(df_selected['OrderDate'], errors='coerce')

analysis_date = df_selected['OrderDate'].max() + pd.Timedelta(days=1)
print(f"📅 Analysis date: {analysis_date.date()}")

LRFM = df_selected.groupby('CustomerID').agg({
    'OrderDate': [lambda x: (analysis_date - x.max()).days,     # Recency (R)
                  lambda x: (x.max() - x.min()).days],           # Length (L)
    'OrderID': 'nunique',                                        # Frequency (F)
    'OrderTotal': 'sum'                                          # Monetary (M)
})

LRFM.columns = ['Recency', 'Length', 'Frequency', 'Monetary']
LRFM = LRFM.reset_index()

print("✅ LRFM Table Sample:")
print(LRFM.head())

📅 Analysis date: 2018-02-01
✅ LRFM Table Sample:
   CustomerID  Recency  Length  Frequency     Monetary
0           1     1074       0          1   472.450012
1           2      286     787          4  1618.660042
2           3       14     991          5  3189.200037
3           4       33    1039          4  1480.709993
4           5      262     826          3  1101.919998


# (3) Phân cụm bằng K-Means

In [ ]:
LRFM = LRFM.dropna()
scaler = MinMaxScaler()
LRFM_scaled = scaler.fit_transform(LRFM[['Recency', 'Length', 'Frequency', 'Monetary']])
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(LRFM_scaled)
LRFM['Cluster'] = clusters

print("Phân cụm K-Means hoàn tất! Dữ liệu mẫu:")
print(LRFM.head())

Phân cụm K-Means hoàn tất! Dữ liệu mẫu:
   CustomerID  Recency  Length  Frequency     Monetary  Cluster
0           1     1074       0          1   472.450012        1
1           2      286     787          4  1618.660042        2
2           3       14     991          5  3189.200037        0
3           4       33    1039          4  1480.709993        0
4           5      262     826          3  1101.919998        2


In [ ]:
customer_segment = df_selected[['CustomerID', 'Segment']].drop_duplicates()
LRFM = pd.merge(LRFM, customer_segment, on='CustomerID', how='left')

In [12]:
print(LRFM.head())

   CustomerID  Recency  Length  Frequency     Monetary  Cluster      Segment
0           1     1074       0          1   472.450012        1     Consumer
1           2      286     787          4  1618.660042        2     Consumer
2           3       14     991          5  3189.200037        0     Consumer
3           4       33    1039          4  1480.709993        0     Consumer
4           5      262     826          3  1101.919998        2  Home Office


In [ ]:
LRFM.to_csv("LRFM_with_segment1.csv", index=False)
LRFM.to_excel("LRFM_with_segment1.xlsx", index=False)
print("✅ Đã xuất file LRFM kèm Cluster + Segment:")

✅ Đã xuất file LRFM kèm Cluster + Segment:


In [ ]:
lr_avg = (
    LRFM.groupby('Cluster')[['Recency', 'Length', 'Frequency', 'Monetary']]
    .mean()
    .round(3)
)

lr_count = LRFM['Cluster'].value_counts().sort_index().rename('Num_Customers')
cluster_summary = pd.concat([lr_count, lr_avg], axis=1).reset_index()

print("📊 Bảng tổng hợp LRFM + số lượng khách:")
print(cluster_summary)


📊 Bảng tổng hợp LRFM + số lượng khách:
   Cluster  Num_Customers  Recency   Length  Frequency  Monetary
0        0           5231  128.560  913.752      6.210  3481.299
1        1           5969  852.257   12.599      1.072   305.145
2        2           6052  286.644  650.269      3.787  1939.514
3        3           3400  287.609   30.669      1.162   377.714


# (4) Phân tích đặc trưng cụm

In [ ]:
def describe_cluster_detail(LRFM, cluster_id):
    cluster_df = LRFM[LRFM['Cluster'] == cluster_id]
    stats = cluster_df[['Length','Recency', 'Frequency', 'Monetary']].describe().T
    stats = stats.rename(columns={
        'count': 'Count (number of customers)',
        'mean': 'Mean',
        'std': 'Standard Deviation',
        'min': 'Min',
        '25%': '25%',
        '50%': '50%',
        '75%': '75%',
        'max': 'Max'
    })
    stats.insert(0, 'Variable', stats.index)
    return stats.reset_index(drop=True).round(3)
cluster_0_stats = describe_cluster_detail(LRFM, cluster_id=3)
print(cluster_0_stats.to_string())

    Variable  Count (number of customers)     Mean  Standard Deviation   Min     25%     50%     75%      Max
0     Length                       3400.0   30.669              83.912  0.00    0.00    0.00    0.00   401.00
1    Recency                       3400.0  287.609             140.896  1.00  168.00  272.00  403.00   578.00
2  Frequency                       3400.0    1.162               0.511  1.00    1.00    1.00    1.00     5.00
3   Monetary                       3400.0  377.714             411.060  8.47   83.56  246.31  442.04  3392.37


# (5) Kiểm định giả thuyết thống kê

In [ ]:
manova = MANOVA.from_formula('Recency + Length + Frequency + Monetary ~ C(Cluster)', data=LRFM)
manova_result = manova.mv_test()
print("📊 Kết quả MANOVA:")
print(manova_result)

📊 Kết quả MANOVA:
                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value  Num DF   Den DF     F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda  0.0356 4.0000 20645.0000 139716.7541 0.0000
         Pillai's trace  0.9644 4.0000 20645.0000 139716.7541 0.0000
 Hotelling-Lawley trace 27.0703 4.0000 20645.0000 139716.7541 0.0000
    Roy's greatest root 27.0703 4.0000 20645.0000 139716.7541 0.0000
--------------------------------------------------------------------
                                                                    
--------------------------------------------------------------------
       C(Cluster)        Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda  0.0194 12.000

# (6) Phân tích vòng đời và giá trị khách hàng

In [ ]:
LRFM['AvgOrderValue'] = LRFM['Monetary'] / LRFM['Frequency']
LRFM['LifetimeYear'] = LRFM['Length'] / 365
LRFM['CLV'] = LRFM['AvgOrderValue'] * LRFM['Frequency'] * LRFM['LifetimeYear']

clv_by_cluster = LRFM.groupby("Cluster")['CLV'].mean().reset_index()
clv_by_cluster = clv_by_cluster.rename(columns={"CLV": "Avg_CLV"})

clv_by_segment = LRFM.groupby("Segment")["CLV"].mean().reset_index()
clv_by_segment = clv_by_segment.rename(columns={"CLV": "Avg_CLV_Segment"})

print("📊 CLV trung bình theo từng cụm khách hàng:")
print(clv_by_cluster)
print("\n")
print("📊 CLV trung bình theo từng Segment:")
print(clv_by_segment)

📊 CLV trung bình theo từng cụm khách hàng:
   Cluster      Avg_CLV
0        0  8622.116420
1        1    36.972006
2        2  3466.455400
3        3    82.791523


📊 CLV trung bình theo từng Segment:
       Segment  Avg_CLV_Segment
0     Consumer      3385.633358
1    Corporate      3090.229713
2  Home Office      2983.906455
